In [74]:
#!/usr/bin/env python3

import os
#from keras.datasets import mnist
import tensorflow as tf
#import numpy as np
from PIL import Image
import os, sys
import glob
import numpy as np

from tensorflow import keras
#import keras
#from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from keras.utils import to_categorical

import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [87]:
'-------------------------------------------------------'
'               EXTERNAL PATHS                          '
'-------------------------------------------------------'
'''path of external device '''
rootDir = "/Volumes/Disk E/"
datasDir = rootDir + "/crohme2016_inkml_datasets/CROHME2016_data/"

imagesDir = rootDir + "/crohme2016_inkml_images/"
'''now get train valid test dir'''

trainDir = datasDir + 'Task-2-Symbols/task2-trainSymb2014/'
trainDirDatasets = trainDir + 'trainingSymbols/'
trainDir_ground_truth = trainDir + 'trainingSymbols/iso_GT.txt'
trainDatasImmagesDir = "/Volumes/Disk E/crohme2016_inkml_images/train_images"

testDir = datasDir + 'Task-2-Symbols/task2-testSymbols2014/'
testDir_ground_truth = testDir + "testSymbols_2016_iso_GT.txt"
testDirDatasets = testDir + "testSymbols/"
testDatasImmagesDir = "/Volumes/Disk E/crohme2016_inkml_images/test_images"

valDir = datasDir + 'Task-2-Symbols/task2-validation-isolatedTest2013b/'
valDirDatasets = datasDir + 'Task-2-Symbols/task2-validation-isolatedTest2013b/'
valDir_ground_truth = valDirDatasets + 'iso_GT.txt'
valDatasImmagesDir = "/Volumes/Disk E/crohme2016_inkml_images/val_images"





In [199]:
class CNN(object):

    differents_characters = 101
    image_len = 28
    X_train_len = 85802
    X_test_len = 18435
    X_val_len = 12504

    def __init__(self):
        self.X_train, self.X_test, self.X_val = [], [], []
        self.y_train, self.y_test, self.y_val = [], [], []

        self.y_train_character_ordered = []
        self.y_test_character_ordered = []
        self.y_val_character_ordered = []
       
        #self.gt = [ {sortedDictTrain}  , {sortedDictTest}, {sortedDictval} ]
        #sortedDictTrain = ')' : [chemin de ttes les images de parenthèses], '9': [chemin de ttes les images de 9],  ...
        self.gt = {} 

        '''first load grounth truth and prevent junk data to load '''
        self.get_all_images_categorical()
        '''second load images and prevent junk images  to load '''
        self.get_all_images_Matrix()

        self.X_train, self.X_test, self.X_val = np.array(self.X_train), np.array(self.X_test), np.array(self.X_val)
        self.y_train, self.y_test, self.y_val = np.array(self.y_train), np.array(self.y_test), np.array(self.y_val)
        

    def get_matrix_from_image(self, infilename):
        img = Image.open(infilename)
        img.load()
        imageMatrix = np.asarray(img, dtype="float32")/255
        return imageMatrix
    #------------------------------------------------------------------------------------------------------#
    #                         GET TRAIN RESAUL
    #-------------------------------------------------------------------------------------------------------#
    def get_which_character(y):
        pass
    #------------------------------------------------------------------------------------------------------#
    # result: X_train = [imageMatrix_iso0.png, imageMatrix_iso0.png, ........., imageMatrix_iso85802.png ]
    #         X_test = [imageMatrix_iso0.png, imageMatrix_iso0.png, ........., imageMatrix_iso18402.png  ]
    #         X_val = [imageMatrix_iso0.png, imageMatrix_iso0.png, ........., imageMatrix_iso18402.png ]
    #--------------------------------------------------------------------------------------------------------#
    def get_all_images_Matrix(self):
        directories = [trainDatasImmagesDir, testDatasImmagesDir, valDatasImmagesDir]
        if not self.gt:
            print("Error no ground truth to prevent to load junk image")
            return
        for index, data_dir_abs_path in enumerate(directories):
            print("loding image ..............\n")
            if os.path.isdir(data_dir_abs_path):
                if index == 0:
                    slice_start, slice_stop = 3, -4
                    ground_truth = self.gt['train_gt']
                if index == 1:
                    slice_start, slice_stop = 3, -4
                    ground_truth = self.gt['test_gt']
                if index == 2:
                    slice_start, slice_stop = 4, -4
                    ground_truth = self.gt['val_gt']

                print(".... start loading  in folder: {}".format(data_dir_abs_path))
                #get array of files
                images_files_array = sorted(glob.glob(data_dir_abs_path + os.sep + "*.png"))
               
                #sort files array
                images_files_array = sorted(images_files_array, key=lambda name: int(
                (os.path.basename(name))[slice_start: slice_stop]))
                
                #inkml_files = inkml_files.sort(key=self.sortGlobFilesArray)
                for j, image_file_abs_path in enumerate(images_files_array):
                    
                    file_name_index = image_file_abs_path.split('/')[-1]
                    file_name_index = file_name_index[int(slice_start): int(slice_stop)]
                    #print(".............fle check: {} .............".format(file_name_index))
                    file_is_junk = True
                    for key in ground_truth:
                        if file_name_index in ground_truth[key]:
                            file_is_junk = False
                    if  False == file_is_junk:
                        imageMatrix = self.get_matrix_from_image(image_file_abs_path).reshape(28, 28, 1)
               
                        if index == 0:
                            self.X_train.append(imageMatrix)
                        if index == 1:
                            self.X_test.append(imageMatrix)
                        if index == 2:
                            self.X_val.append(imageMatrix)
                    #else: 
                        #print('...............Found JUNK FILE .....................\n')
                if index == 0:
                    print("------- LEN IMAGES FILES: ", len(self.X_train))
                if index == 1:
                    print("------- LEN IMAGES FILES: ", len(self.X_test))
                if index == 2:
                    print("------- LEN IMAGES FILES: ", len(self.X_val))

            else:
                print("Error: External Devices not found\n")
        
        
    #---------------------------------------------------------------------------------------------------#
    #  Result: y_train = Matrix 101 x 101, y_test = Matrix 101 x 101, y_val = Matrix 101 x 101
    #---------------------------------------------------------------------------------------------------#
    def get_all_images_categorical(self):
        gt_directories_files = [trainDir_ground_truth, testDir_ground_truth, valDir_ground_truth]
        imagesDir = [trainDatasImmagesDir, testDatasImmagesDir, valDatasImmagesDir]
        
        for index, abs_gt_file_path in enumerate(gt_directories_files):
            #print("Start reading grount truth ..... {}" .format(os.path.basename(abs_gt_file_path)))
            sorted_gt, total_character, y = self.read_gt(abs_gt_file_path, imagesDir[index])
            
            length = len(sorted_gt.keys())
            
            for i, el in enumerate(y):
                x = np.zeros(length, dtype=np.float32)
                x[el] = 1.0
                        
                if index == 0:
                    self.y_train.append(x)
                    self.gt['train_gt'] = sorted_gt
                if index == 1:
                    self.y_test.append(x)
                    self.gt['test_gt'] = sorted_gt
                if index == 2:
                    self.y_val.append(x)
                    self.gt['val_gt'] = sorted_gt
    
    
    #------------------------------------------------------------------------------#
    # return a Dictionary
    # {
    # ')' : [abs_path_image1, abs_path_imag2, ...]
    #    .....................................
    #   '!'  : [abs_path_image14, abs_path_imag267, ...]
    # }
    #------------------------------------------------------------------------------#
    def read_gt(self, abs_gt_file_path, imagesDir):
        '''Loads the routes of the png files'''
        try:
            fileName = os.path.basename(abs_gt_file_path)
            print("reading ground truth {} ..................".format(fileName))
            with open(abs_gt_file_path) as file:
                lines = file.readlines()

                aDict = {}
                aLabelsSet = []
                total_character = 0
                all_labels = []
                
                for index, line in enumerate(lines):
                    line = line.strip()

                    parts = line.split(',')
                    label = parts[1].strip()

                    data = parts[0].split('_')
                    index = data[-1].strip()
                    if(label != 'junk'):
                        all_labels.append(label)
                    if label not in aLabelsSet and label != 'junk':
                        aLabelsSet.append(label)

                    if label in aDict and label != 'junk':
                        #aDict[label].append(imagesDir + '/' + 'iso' + index + '.png')
                        aDict[label].append(index)
                    else:
                        #aDict[label] = [imagesDir + '/' + 'iso' + index + '.png']
                        aDict[label] = [index]
                    total_character = total_character + 1

            #sort dictionary but others method doesn't work
            bDict = {}
            y = []
            for label in aLabelsSet:
                bDict[label] = aDict[label]
            for label in all_labels:
                for i, el in enumerate(aLabelsSet):
                    if(el == label):
                        y.append(i)

            #aDict = OrderedDict(sorted(aDict.items(), key=lambda t: t[0]))
            # print("dictionnary sorted keys : ", bDict.keys())
            print("Finish readin ground {} truth: {}".format(fileName, len(bDict)))
            #print("bDict length:........", len(bDict))
           
            return bDict, total_character,y

        except FileNotFoundError as e:
            print(e)
    
    def create_model(self):
        model = keras.Sequential()
        #adding layers
        model.add(keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28, 28, 1)))
        model.add(keras.layers.MaxPooling2D(pool_size=2))
        model.add(keras.layers.Dropout(0.3))
        model.add(keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
        model.add(keras.layers.MaxPooling2D(pool_size=2))
        model.add(keras.layers.Dropout(0.3))
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(256, activation='relu'))
        model.add(keras.layers.Dropout(0.5))
        model.add(keras.layers.Dense(101, activation='softmax'))
        return model

    def compile_model(self):
        #compile model using accuracy to measure model performance
        # Compile the model
        model.compile(
            loss=keras.losses.mean_squared_error, optimizer='sgd')
        return model

    def train_model(self):
        #fit the model
        model.fit(self.X_train, self.y_train, batch_size=500, epochs=10)
        # Evaluate the model on test set
        score=model.evaluate(self.X_test, self.y_test, verbose=0)
        # Print test accuracy
        print('\nTest accuracy: {}'.format(score[1]))
        return model
       
    


In [200]:

    cnn = CNN()

   # print("X_train_shape:\n", cnn.X_train.shape)
   # print("X_test_shape:\n", cnn.X_test.shape)
   # print("X_val_shape:\n" ,cnn.X_val.shape)

   # print("Y_train_shape:\n", cnn.y_train.shape)
   # print("Y_test_shape:\n" ,cnn.y_test.shape)
   # print("Y_val_shape:\n", cnn.y_val.shape)'

    #print("X_train_keys:\n", cnn.y_train_character_ordered)
    #print("X_test_keys:\n", cnn.y_test_character_ordered)
    #print("X_val_keys:\n", cnn.y_val_character_ordered)'''
    
    #print("X_train_0\n", cnn.X_train[0])
    #print("y_train_0\n", cnn.y_train[0])
    
    #print("X_test_0\n", cnn.X_test[0])
    #print("y_tesy_0\n", cnn.y_test[0])
    
    #print("X_val_0\n", cnn.X_val[0])
    #print("y_val_0\n", cnn.y_val[0])

    #model = cnn.create_model()
    #model = cnn.compile_model()
    #model = cnn.train_model()

reading ground truth iso_GT.txt ..................
Finish readin ground iso_GT.txt truth: 101
reading ground truth testSymbols_2016_iso_GT.txt ..................
Finish readin ground testSymbols_2016_iso_GT.txt truth: 101
reading ground truth iso_GT.txt ..................
Finish readin ground iso_GT.txt truth: 101
loding image ..............

.... start loading  in folder: /Volumes/Disk E/crohme2016_inkml_images/train_images
------- LEN IMAGES FILES:  85802
loding image ..............

.... start loading  in folder: /Volumes/Disk E/crohme2016_inkml_images/test_images
...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FI

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FI


...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK F

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FI

...

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FI

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FI

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FI

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FI

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FILE .....................

...............Found JUNK FI

In [181]:
cnn.X_train.shape

(85802, 28, 28, 1)

In [182]:
cnn.y_train.shape

(85802, 101)

In [195]:
cnn.y_train[62]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [ ]:
model = cnn.create_model()
model = cnn.compile_model()
model = cnn.train_model()

Train on 85802 samples
Epoch 1/100
85802/85802 [==============================] - 73s 845us/sample - loss: 0.0098
Epoch 2/100
85802/85802 [==============================] - 73s 856us/sample - loss: 0.0098
Epoch 3/100
85802/85802 [==============================] - 72s 841us/sample - loss: 0.0098
Epoch 4/100
85802/85802 [==============================] - 71s 833us/sample - loss: 0.0098
Epoch 5/100
85802/85802 [==============================] - 77s 894us/sample - loss: 0.0098
Epoch 6/100
85802/85802 [==============================] - 74s 868us/sample - loss: 0.0098
Epoch 7/100
56000/85802 [==================>...........] - ETA: 25s - loss: 0.0098